In [1]:
import pandas as pd
import os
import re

articles_csv='data/misq_1977_2017_march.csv'
articles_file='articles.txt'
authors_file='authors.txt'
keywords_file='keywords.txt'
try:
    os.remove(authors_file)
except OSError:
    pass
try:
    os.remove(articles_file)
except OSError:
    pass
try:
    os.remove(keywords_file)
except OSError:
    pass

df=pd.read_csv(articles_csv,dtype='str')
df.fillna('',inplace=True)
articles_file=open(articles_file,'a')
authors_file=open(authors_file,'a')
keywords_file=open(keywords_file,'a')
all_authors=[]
all_keywords=[]

def addAuthor(author_name):
    if author_name in all_authors:
		return
    all_authors.append(author_name)    	
    text='''    
---start---
\'\'\'this_title\'\'\'
{{author papers}}
[[Category:author]]
---end---
'''
    text= text.replace('this_title',author_name)
    authors_file.write(text)
    

def addKeyword(keyword):
    if keyword in all_keywords:
		return
    all_keywords.append(keyword)    	
    text='''
---start---
\'\'\'this_title\'\'\'
{{keyword papers}}
[[Category:keyword]]
---end---
'''
    text= text.replace('this_title',keyword)
    keywords_file.write(text)
    

def addArticle(paper):
    authors_list=paper['authors'].split(';') 
    i=0
    authors_text=''
    for author in authors_list:
        author_name_split=reversed(author.strip().split(','))
        author_name=' '.join(name.strip() for name in author_name_split)
        addAuthor(author_name)
        authors_text=authors_text+ author_name +','
    keywords_list=paper['keywords'].split(';') 
    i=0
    keywords_text=''
    for keyword in keywords_list:
        keyword=keyword.strip()
        if keyword != '':
            addKeyword(keyword)
            keywords_text=keywords_text+ keyword+','
        
    text=''' 
---start---
\'\'\'this_title\'\'\'
{{header}}
{{article
|author= this_author
|source= this_source
|year= this_year
|volume=this_volume
|issue=this_issue
|page=this_bpage-this_epage
|abstract = this_abstract
|keyword = this_keyword
}}
{{topics}}
{{theories}}
{{methods}}
{{footer}}
[[Category:article]]
---end---
'''
    text= text.replace('this_title',paper['title'])
    text= text.replace('this_author',authors_text)
    text= text.replace('this_source',paper['source'])
    text= text.replace('this_year',paper['year'])
    text= text.replace('this_volume',paper['volume'])
    text= text.replace('this_issue',paper['issue'])
    text= text.replace('this_bpage',paper['bpage'])
    text= text.replace('this_epage',paper['epage'])
    text= text.replace('this_abstract',paper['abstract'])
    text= text.replace('this_keyword',keywords_text)
    articles_file.write(text)




In [3]:
for i,row in df.iterrows():
    if i < 0 : break
    paper={
    'title':row['TI'],
    'authors':row['AF'],
    'source':row['SO'],
    'keywords':row['DE'],
    'abstract':row['AB'],
    'year':row['PY'],
    'volume':row['VL'],
    'issue':row['IS'],
    'bpage':row['BP'],
    'epage':row['EP'],
        }
    print paper['title']

    addArticle(paper)

1985 Opinion Survey of MIS Managers:  Key Issues
A Balanced Orientation for the Information Systems Manager
A Behavioral Analysis for Planning MIS Implementation
A Company/University Joint Venture to Build a Knowledge-Based System
A Comparative Examination of Systems Analysis Techniques
A Comparison of Judgement, Skills, and Prompting Effects Between Auditors and Systems Analysts
A Comprehensive Conceptualization of Post-Adoptive Behaviors Associated with Information Technology Enabled Work Systems
A Confessional Account of an Ethnography About Knowledge Work
A Configural Approach to Coordinating Expertise in Software Development Teams
A Confirmatory Factor Analysis of the End-User Computing Satisfaction Instrument
A Contingency Model for User Involvement in DSS Development
A Contingency View of Managing the Data Processing Organization
A Cost-Based Database Request Distribution Technique for Online e-Commerce Applications
A Cross-Cultural Study on Escalation of Commitment Behavior in 

In [4]:
articles_file.close()
authors_file.close()
keywords_file.close()